In [1]:
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import sys
import itertools
from collections import namedtuple
from pathlib import Path

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

%load_ext autoreload
%autoreload 1
%aimport analyze

with open('plots/style.json') as f:
    mpl.rcParams.update(json.load(f))

In [2]:
summit = analyze.Analysis('results/summit')
print(summit)

Analysis for Celeritas v0.1.2+fb3f5506 on summit


In [3]:
# Obsolete:

#avg_per_iter = analyze.groupby_notinstance(
#    summit.result['num_steps'] / summit.result['num_step_iters']
#).mean()
#occupancy = avg_per_iter / summit.input['max_num_tracks']

In [15]:
summed = analyze.summarize_instances(summit.result[[
    'avg_steps_per_primary',
    'avg_time_per_primary',
    'avg_time_per_step',
    'num_steps',
    'total_time',
    'unconverged',
    'slot_occupancy']])

In [11]:
rel_err = summed.xs('std', axis=1, level=1) / summed.xs('mean', axis=1, level=1)
high_err = rel_err > 0.02
rel_err[high_err].dropna(how='all').dropna(how='all', axis=1)

avg_time_per_primary  avg_time_per_step  \
problem              geo     arch                                            
cms2018              vecgeom gpu               0.032770           0.032485   
simple-cms+field+msc vecgeom cpu               0.055442           0.052778   
                             gpu               0.487321           0.487208   
simple-cms+msc       orange  gpu               0.083089           0.083043   
testem15+field       orange  gpu               0.132725           0.132779   
testem15+field+msc   orange  gpu               0.134800           0.134767   
                     vecgeom gpu               0.116093           0.116124   
testem3-flat         orange  gpu               0.083192           0.083131   
                     vecgeom cpu               0.027352           0.027381   
                             gpu               0.059092           0.058964   
testem3-flat+field   orange  gpu               0.075610           0.075662   

                                   total_time  unconverged  
problem              geo     arch                           
cms2018              vecgeom gpu     0.032770          NaN  
simple-cms+field+msc vecgeom cpu     0.055442     0.918937  
                             gpu     0.487321     0.271586  
simple-cms+msc       orange  gpu     0.083089          NaN  
testem15+field       orange  gpu     0.132725          NaN  
testem15+field+msc   orange  gpu     0.134800          NaN  
                     vecgeom gpu     0.116093          NaN  
testem3-flat         orange  gpu     0.083192          NaN  
                     vecgeom cpu     0.027352          NaN  
                             gpu     0.059092          NaN  
testem3-flat+field   orange  gpu     0.075610          NaN

In [6]:
def get_cpugpu_ratio(mean):
    mean = mean.unstack()
    return mean['cpu'] / mean['gpu']

In [12]:
get_cpugpu_ratio(summed[('avg_time_per_step', 'mean')])

problem               geo    
cms2018               vecgeom     9.257449
cms2018+field+msc     vecgeom     6.522801
simple-cms+field      orange           NaN
simple-cms+field+msc  orange           NaN
                      vecgeom     6.441743
simple-cms+msc        orange     21.553550
testem15              orange     20.930640
testem15+field        orange     25.222209
testem15+field+msc    orange     30.617715
                      vecgeom    27.996433
testem3-flat          orange     26.565617
                      vecgeom    22.603086
testem3-flat+field    orange     34.072787
testem3-flat+msc      orange     27.976321
dtype: float64

In [18]:
speedup = get_cpugpu_ratio(summed[('total_time', 'mean')])

In [19]:
speedup.apply("{:.1f}".format)

problem               geo    
cms2018               vecgeom     9.3
cms2018+field+msc     vecgeom    23.9
simple-cms+field      orange      nan
simple-cms+field+msc  orange      nan
                      vecgeom     6.5
simple-cms+msc        orange     21.6
testem15              orange     20.9
testem15+field        orange     25.2
testem15+field+msc    orange     30.6
                      vecgeom    28.0
testem3-flat          orange     26.6
                      vecgeom    22.6
testem3-flat+field    orange     34.1
testem3-flat+msc      orange     28.0
dtype: object

In [17]:
summed['slot_occupancy']

count      mean       std
problem              geo     arch                           
cms2018              vecgeom cpu     6.0  0.992314  0.001246
                             gpu     6.0  0.187132  0.017699
cms2018+field+msc    vecgeom cpu     6.0  1.000000  0.000000
                             gpu     6.0  0.272762  0.000258
simple-cms+field     orange  cpu     0.0       NaN       NaN
                             gpu     0.0       NaN       NaN
simple-cms+field+msc orange  cpu     0.0       NaN       NaN
                             gpu     0.0       NaN       NaN
                     vecgeom cpu     6.0  0.134025  0.029934
                             gpu     6.0  0.117174  0.000048
simple-cms+msc       orange  cpu     6.0  0.998779  0.000240
                             gpu     6.0  0.609701  0.024622
testem15             orange  cpu     6.0  0.998714  0.000277
                             gpu     6.0  0.654161  0.009497
testem15+field       orange  cpu     6.0  0.998859  0.000262
                             gpu     6.0  0.657536  0.015953
testem15+field+msc   orange  cpu     6.0  0.998767  0.000103
                             gpu     6.0  0.660212  0.004414
                     vecgeom cpu     6.0  0.998767  0.000103
                             gpu     6.0  0.660212  0.004414
testem3-flat         orange  cpu     6.0  0.998949  0.000154
                             gpu     6.0  0.706494  0.011048
                     vecgeom cpu     6.0  0.998949  0.000154
                             gpu     6.0  0.706494  0.011048
testem3-flat+field   orange  cpu     2.0  0.998670  0.000197
                             gpu     5.0  0.709734  0.008457
testem3-flat+msc     orange  cpu     4.0  0.998937  0.000169
                             gpu     4.0  0.677107  0.021935

In [ ]:
(fig, ax) = plt.subplots()
plot_counts(ax, full);

## Crusher

In [20]:
crusher = analyze.Analysis('results/crusher')
print(crusher)

Analysis for Celeritas v0.1.2+fb3f5506 on crusher


In [27]:
csum = analyze.summarize_instances(crusher.result[~crusher.invalid][[
    'avg_steps_per_primary',
    'avg_time_per_step',
    'num_steps',
    'total_time',
]])

In [29]:
rel_err = csum.xs('std', axis=1, level=1) / csum.xs('mean', axis=1, level=1)
high_err = rel_err > 0.02
rel_err[high_err].dropna(how='all').dropna(how='all', axis=1)

,,
problem,geo,arch


In [31]:
get_cpugpu_ratio(csum[('total_time', 'mean')]).apply("{:.1f}".format)

problem             geo   
simple-cms+msc      orange    16.9
testem15            orange    15.4
testem15+field      orange    16.3
testem15+field+msc  orange    17.0
testem3-flat        orange    13.7
testem3-flat+field  orange    14.1
testem3-flat+msc    orange    17.9
dtype: object